### Fine-tuning a model for music classification

In this section, we’ll present a step-by-step guide on fine-tuning an encoder-only transformer model for music classification. We’ll use a lightweight model for this demonstration and fairly small dataset, meaning the code is runnable end-to-end on any consumer grade GPU, including the T4 16GB GPU provided in the Google Colab free tier. The section includes various tips that you can try should you have a smaller GPU and encounter memory issues along the way.

#### The Dataset
To train our model, we’ll use the GTZAN dataset, which is a popular dataset of 1,000 songs for music genre classification. Each song is a 30-second clip from one of 10 genres of music, spanning disco to metal. We can get the audio files and their corresponding labels from the Hugging Face Hub with the load_dataset() function from 🤗 Datasets:

In [1]:
from datasets import load_dataset

gtzan = load_dataset("marsyas/gtzan", "all")
gtzan

README.md:   0%|          | 0.00/4.42k [00:00<?, ?B/s]

gtzan.py:   0%|          | 0.00/3.35k [00:00<?, ?B/s]

genres.tar.gz:   0%|          | 0.00/1.23G [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 999
    })
})

GTZAN doesn’t provide a predefined validation set, so we’ll have to create one ourselves. The dataset is balanced across genres, so we can use the train_test_split() method to quickly create a 90/10 split as follows:

In [2]:
gtzan = gtzan["train"].train_test_split(seed=42, shuffle=True, test_size=0.1)
gtzan

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 899
    })
    test: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 100
    })
})

In [3]:
gtzan["train"][0]

{'file': '/Users/ganeshnagaraja/.cache/huggingface/datasets/downloads/extracted/5e39b16be2f5fd98fbc6ba913e6a0bb5f3e1d89dbc32c95cb26f9027fd4087fe/genres/pop/pop.00098.wav',
 'audio': {'path': '/Users/ganeshnagaraja/.cache/huggingface/datasets/downloads/extracted/5e39b16be2f5fd98fbc6ba913e6a0bb5f3e1d89dbc32c95cb26f9027fd4087fe/genres/pop/pop.00098.wav',
  'array': array([ 0.10720825,  0.16122437,  0.28585815, ..., -0.22924805,
         -0.20629883, -0.11334229]),
  'sampling_rate': 22050},
 'genre': 7}

In [4]:
id2label_fn = gtzan["train"].features["genre"].int2str
id2label_fn(gtzan["train"][0]["genre"])

'pop'

As we saw in Unit 1, the audio files are represented as 1-dimensional NumPy arrays, where the value of the array represents the amplitude at that timestep. For these songs, the sampling rate is 22,050 Hz, meaning there are 22,050 amplitude values sampled per second. We’ll have to keep this in mind when using a pretrained model with a different sampling rate, converting the sampling rates ourselves to ensure they match. We can also see the genre is represented as an integer, or class label, which is the format the model will make it’s predictions in. Let’s use the int2str() method of the genre feature to map these integers to human-readable names:

In [5]:
import gradio as gr


def generate_audio():
    example = gtzan["train"].shuffle()[0]
    audio = example["audio"]
    return (
        audio["sampling_rate"],
        audio["array"],
    ), id2label_fn(example["genre"])


with gr.Blocks() as demo:
    with gr.Column():
        for _ in range(4):
            audio, label = generate_audio()
            output = gr.Audio(audio, label=label)

demo.launch(debug=True)

/Users/ganeshnagaraja/miniconda3/envs/LLM/lib/python3.10/site-packages/gradio/processing_utils.py:749: UserWarning: Trying to convert audio automatically from float64 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


#### Picking a pretrained model for audio classification
To get started, let’s pick a suitable pretrained model for audio classification. In this domain, pretraining is typically carried out on large amounts of unlabeled audio data, using datasets like LibriSpeech https://huggingface.co/datasets/librispeech_asr  and Voxpopuli https://huggingface.co/datasets/facebook/voxpopuli . The best way to find these models on the Hugging Face Hub is to use the “Audio Classification” filter, as described in the previous section. Although models like Wav2Vec2 and HuBERT are very popular, we’ll use a model called DistilHuBERT. This is a much smaller (or distilled) version of the HuBERT https://huggingface.co/docs/transformers/model_doc/hubert model, which trains around 73% faster, yet preserves most of the performance.

### From audio to machine learning features

#### Preprocessing the data

Similar to tokenization in NLP, audio and speech models require the input to be encoded in a format that the model can process. In 🤗 Transformers, the conversion from audio to the input format is handled by the feature extractor of the model. Similar to tokenizers, 🤗 Transformers provides a convenient AutoFeatureExtractor class that can automatically select the correct feature extractor for a given model. To see how we can process our audio files, let’s begin by instantiating the feature extractor for DistilHuBERT from the pre-trained checkpoint:

In [7]:
from transformers import AutoFeatureExtractor

model_id = "ntu-spml/distilhubert"
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True, return_attention_mask=True
)

preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

Since the sampling rate of the model and the dataset are different, we’ll have to resample the audio file to 16,000 Hz before passing it to the feature extractor. We can do this by first obtaining the model’s sample rate from the feature extractor:

In [8]:
sampling_rate = feature_extractor.sampling_rate
sampling_rate

16000

Next, we resample the dataset using the cast_column() method and Audio feature from 🤗 Datasets:

In [9]:
from datasets import Audio

gtzan = gtzan.cast_column("audio", Audio(sampling_rate=sampling_rate))

In [10]:
gtzan["train"][0]

{'file': '/Users/ganeshnagaraja/.cache/huggingface/datasets/downloads/extracted/5e39b16be2f5fd98fbc6ba913e6a0bb5f3e1d89dbc32c95cb26f9027fd4087fe/genres/pop/pop.00098.wav',
 'audio': {'path': '/Users/ganeshnagaraja/.cache/huggingface/datasets/downloads/extracted/5e39b16be2f5fd98fbc6ba913e6a0bb5f3e1d89dbc32c95cb26f9027fd4087fe/genres/pop/pop.00098.wav',
  'array': array([ 0.08735093,  0.20183384,  0.47908676, ..., -0.1874318 ,
         -0.23294398, -0.13517429]),
  'sampling_rate': 16000},
 'genre': 7}

Great! We can see that the sampling rate has been downsampled to 16kHz. The array values are also different, as we’ve now only got approximately one amplitude value for every 1.5 that we had before.

A defining feature of Wav2Vec2 and HuBERT like models is that they accept a float array corresponding to the raw waveform of the speech signal as an input. This is in contrast to other models, like Whisper, where we pre-process the raw audio waveform to spectrogram format.

We mentioned that the audio data is represented as a 1-dimensional array, so it’s already in the right format to be read by the model (a set of continuous inputs at discrete time steps). So, what exactly does the feature extractor do?

Well, the audio data is in the right format, but we’ve imposed no restrictions on the values it can take. For our model to work optimally, we want to keep all the inputs within the same dynamic range. This is going to make sure we get a similar range of activations and gradients for our samples, helping with stability and convergence during training.

To do this, we normalise our audio data, by rescaling each sample to zero mean and unit variance, a process called feature scaling. It’s exactly this feature normalisation that our feature extractor performs!

We can take a look at the feature extractor in operation by applying it to our first audio sample. First, let’s compute the mean and variance of our raw audio data:

In [11]:
import numpy as np

sample = gtzan["train"][0]["audio"]

print(f"Mean: {np.mean(sample['array']):.3}, Variance: {np.var(sample['array']):.3}")

Mean: 0.000185, Variance: 0.0493


We can see that the mean is close to zero already, but the variance is closer to 0.05. If the variance for the sample was larger, it could cause our model problems, since the dynamic range of the audio data would be very small and thus difficult to separate. Let’s apply the feature extractor and see what the outputs look like:

In [12]:
inputs = feature_extractor(sample["array"], sampling_rate=sample["sampling_rate"])

print(f"inputs keys: {list(inputs.keys())}")

print(
    f"Mean: {np.mean(inputs['input_values']):.3}, Variance: {np.var(inputs['input_values']):.3}"
)

inputs keys: ['input_values', 'attention_mask']
Mean: -7.01e-09, Variance: 1.0


Alright! Our feature extractor returns a dictionary of two arrays: input_values and attention_mask. The input_values are the preprocessed audio inputs that we’d pass to the HuBERT model. The attention_mask is used when we process a batch of audio inputs at once - it is used to tell the model where we have padded inputs of different lengths.

We can see that the mean value is now very much closer to zero, and the variance bang-on one! This is exactly the form we want our audio samples in prior to feeding them to the HuBERT model.

Note how we’ve passed the sampling rate of our audio data to our feature extractor. This is good practice, as the feature extractor performs a check under-the-hood to make sure the sampling rate of our audio data matches the sampling rate expected by the model. If the sampling rate of our audio data did not match the sampling rate of our model, we’d need to up-sample or down-sample the audio data to the correct sampling rate.

Great, so now we know how to process our resampled audio files, the last thing to do is define a function that we can apply to all the examples in the dataset. Since we expect the audio clips to be 30 seconds in length, we’ll also truncate any longer clips by using the max_length and truncation arguments of the feature extractor as follows:

In [13]:
max_duration = 30.0


def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
        return_attention_mask=True,
    )
    return inputs

With this function defined, we can now apply it to the dataset using the map() method. The .map() method supports working with batches of examples, which we’ll enable by setting batched=True. The default batch size is 1000, but we’ll reduce it to 100 to ensure the peak RAM stays within a sensible range for Google Colab’s free tier:

In [14]:
gtzan_encoded = gtzan.map(
    preprocess_function,
    remove_columns=["audio", "file"],
    batched=True,
    batch_size=100,
    num_proc=1,
)
gtzan_encoded

Map:   0%|          | 0/899 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['genre', 'input_values', 'attention_mask'],
        num_rows: 899
    })
    test: Dataset({
        features: ['genre', 'input_values', 'attention_mask'],
        num_rows: 100
    })
})

If you exhaust your device's RAM executing the above code, you can adjust the batch parameters to reduce the peak RAM usage. In particular, the following two arguments can be modified: * `batch_size`: defaults to 1000, but set to 100 above. Try reducing by a factor of 2 again to 50 * `writer_batch_size`: defaults to 1000. Try reducing it to 500, and if that doesn't work, then reduce it by a factor of 2 again to 250